In [1]:
!conda install -c conda-forge geopy --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [2]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         868 KB

The following NEW packages will be INSTALLED:

    altair:  3.2.0-py36_0 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge


branca-0.3.1         | 25 KB     | #################################

## Introduction

The amount of crime in India is rising everyday. Crimes are categorized into multiple different groups. Our goal in this project is to figure out which states in the country has the most number of a certain crime. We finally conclude with which state has the most overall crimes. This data is useful to agencies that fight crime and could help them figure out which parts of the country require higher and more vigilant security etc. Based on previous data, we can b

## Data

The dataset is take from Kaggle. It contains information about the trials of violent crimes by courts and will eventually help determine the kind of results we watn. Foursquare data is leveraged in order to be able to plot our final results on a map.

In [5]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.


body = client_55b2bb94579d4d9ba04aaf80a8aad6ca.get_object(Bucket='couseracapstone-donotdelete-pr-3vhu2vlrhbzpzg',Key='28_Trial_of_violent_crimes_by_courts.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

crimedf = pd.read_csv(body)
crimedf.head()


,Area_Name,Year,Group_Name,Sub_Group_Name,Trial_of_Violent_Crimes_by_Courts_By_Confession,Trial_of_Violent_Crimes_by_Courts_By_trial,Trial_of_Violent_Crimes_by_Courts_Total
0,Andhra Pradesh,2001,TVC- Arson,10. Arson,20.0,517.0,537.0
1,Arunachal Pradesh,2001,TVC- Arson,10. Arson,0.0,3.0,3.0
2,Assam,2001,TVC- Arson,10. Arson,5.0,142.0,147.0
3,Bihar,2001,TVC- Arson,10. Arson,0.0,208.0,208.0
4,Chandigarh,2001,TVC- Arson,10. Arson,0.0,3.0,3.0


In [6]:
crimedf.shape

(4473, 7)

In [7]:
crimedf.columns=['State', 'Year', 'Group', 'Sub_Group', 'Confession', 'Trial', 'Total']

In [8]:
crimedf.head(10)

,State,Year,Group,Sub_Group,Confession,Trial,Total
0,Andhra Pradesh,2001,TVC- Arson,10. Arson,20.0,517.0,537.0
1,Arunachal Pradesh,2001,TVC- Arson,10. Arson,0.0,3.0,3.0
2,Assam,2001,TVC- Arson,10. Arson,5.0,142.0,147.0
3,Bihar,2001,TVC- Arson,10. Arson,0.0,208.0,208.0
4,Chandigarh,2001,TVC- Arson,10. Arson,0.0,3.0,3.0
5,Chhattisgarh,2001,TVC- Arson,10. Arson,22.0,41.0,63.0
6,Dadra & Nagar Haveli,2001,TVC- Arson,10. Arson,NaN,2.0,2.0
7,Delhi,2001,TVC- Arson,10. Arson,18.0,75.0,93.0
8,Goa,2001,TVC- Arson,10. Arson,NaN,41.0,41.0
9,Gujarat,2001,TVC- Arson,10. Arson,2.0,228.0,230.0


In [9]:
crimedf.dropna(axis=0, inplace=True)
crimedf.shape

(3138, 7)

In [10]:
crimedf.drop(labels=['Group'], axis=1, inplace=True)
crimedf.shape

(3138, 6)

In [11]:
clist=crimedf.values.tolist()

In [20]:
#remove very small number of crimes
for i in range(len(clist)):
    if clist[i][5]<10:
        del clist[i]

In [21]:
len(clist)

2436

In [35]:
for i in range(len(clist)):
    if clist[i][2]=='12. Total Trials (Sum of 1-11 Above)':
        del clist[i]

In [36]:
len(clist)

2185

In [37]:
crimedf=pd.DataFrame(clist)
crimedf.columns=['State', 'Year', 'group', 'Confession', 'Trial', 'Total']
crimedf.head()

,State,Year,group,Confession,Trial,Total
0,Andhra Pradesh,2001,10. Arson,20.0,517.0,537.0
1,Assam,2001,10. Arson,5.0,142.0,147.0
2,Bihar,2001,10. Arson,0.0,208.0,208.0
3,Chhattisgarh,2001,10. Arson,22.0,41.0,63.0
4,Delhi,2001,10. Arson,18.0,75.0,93.0


In [41]:
crimedf.groupby(['State', 'Year']).sum()

Confession    Trial    Total
State             Year                              
Andhra Pradesh    2001        58.0   9246.0   9304.0
                  2002         2.0  10138.0  10140.0
                  2003         0.0   8808.0   8808.0
                  2004         0.0   9825.0   9825.0
                  2005         0.0   9318.0   9318.0
                  2006         0.0   9946.0   9946.0
                  2007         3.0  10148.0  10151.0
                  2008         0.0   9047.0   9047.0
                  2009         2.0   9421.0   9423.0
                  2010        11.0  10523.0  10534.0
Arunachal Pradesh 2001         0.0     48.0     48.0
                  2003         0.0     20.0     20.0
                  2004         0.0     12.0     12.0
                  2005         0.0     92.0     92.0
                  2006         0.0     37.0     37.0
                  2007         0.0     19.0     19.0
                  2008         0.0     22.0     22.0
                  2009         0.0     21.0     21.0
                  2010         0.0     33.0     33.0
Assam             2001       192.0   4223.0   4415.0
                  2002       117.0   3638.0   3755.0
                  2003       122.0   4424.0   4546.0
                  2004       211.0   5139.0   5350.0
                  2005       177.0   4804.0   4981.0
                  2006       227.0   5224.0   5451.0
                  2007       313.0   5628.0   5941.0
                  2008       225.0   4051.0   4276.0
                  2009       200.0   3330.0   3530.0
                  2010       265.0   3559.0   3824.0
Bihar             2001        17.0  14112.0  14129.0
...                            ...      ...      ...
Tripura           2004         0.0     17.0     17.0
                  2006         0.0    166.0    166.0
                  2007         0.0     17.0     17.0
                  2009        12.0    356.0    368.0
                  2010         6.0     89.0     95.0
Uttar Pradesh     2001        82.0  23949.0  24031.0
                  2002        29.0  13457.0  13486.0
                  2003        39.0   8878.0   8917.0
                  2004        20.0  25250.0  25270.0
                  2005         0.0  24116.0  24116.0
                  2006         0.0  24745.0  24745.0
                  2007         0.0   5699.0   5699.0
                  2008         0.0   2817.0   2817.0
                  2009         0.0   2970.0   2970.0
                  2010         0.0   5976.0   5976.0
Uttarakhand       2001        58.0    664.0    722.0
                  2002        48.0    584.0    632.0
                  2003        34.0    900.0    934.0
                  2006         0.0    410.0    410.0
                  2007         0.0    235.0    235.0
West Bengal       2001         9.0   4872.0   4881.0
                  2002         0.0   6551.0   6551.0
                  2003        33.0   7004.0   7037.0
                  2004        61.0   5009.0   5070.0
                  2005        23.0   5365.0   5388.0
                  2006        38.0   9584.0   9622.0
                  2007         8.0   7112.0   7120.0
                  2008         0.0   8011.0   8011.0
                  2009        46.0   6019.0   6065.0
                  2010        51.0   5675.0   5726.0

[260 rows x 3 columns]